## Referencce website 
code from https://blog.csdn.net/dgvv4/article/details/122023047

### **Import library**

In [1]:
import cv2

In [2]:
import mediapipe as mp

In [3]:
import time

In [4]:
import csv
import os

In [5]:
import sqlite3

In [6]:
from datetime import datetime

In [7]:
#to play music
!pip install pygame
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.8.17)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [8]:
!pip install keyboard
import keyboard

### **Create database**

In [9]:
#conn = sqlite3.connect('hand_coordinates.db')

In [10]:
# 创建一个表
#c = conn.cursor()
# 创建用于保存归一化坐标的表
#c.execute('''
#CREATE TABLE IF NOT EXISTS normalized_coordinates (
    #id INTEGER PRIMARY KEY AUTOINCREMENT,
    #timestamp TEXT,
    #x REAL,
    #y REAL
#);
#''')

# 创建用于保存像素坐标的表
#c.execute('''
#CREATE TABLE IF NOT EXISTS pixel_coordinates (
   # id INTEGER PRIMARY KEY AUTOINCREMENT,
   # timestamp TEXT,
   # x INTEGER,
   # y INTEGER
#);
#''')
#conn.commit()

### **Visualize hand landmarks and connections**

In [9]:
cap = cv2.VideoCapture(0)#capture video, 0 means built-in webcam

In [10]:
mpHands = mp.solutions.hands

In [11]:
hands = mpHands.Hands(static_image_mode=False, #静态追踪，低于0.5置信度会再一次跟踪
                      max_num_hands=2, # 最多有2只手
                      min_detection_confidence=0.5, # 最小检测置信度
                      min_tracking_confidence=0.5)  # 最小跟踪置信度

In [12]:
mpDraw = mp.solutions.drawing_utils

In [13]:
pTime = 0 #处理一张图像前的时间
cTime = 0 #一张图处理完的时间

In [14]:
# 存放坐标信息
lmList = []
captured_coordinates = []

In [17]:
pygame.init()
pygame.mixer.init()
pygame.mixer.music.load(r'C:\Users\OS\Desktop\Final_year_project\Prototype1_and_dataset\Code_project_construction2.2_LSTMplusGesture\datasets\mozart\mz_311_1.mid')
music_files = [f for f in os.listdir(music_directory) if f.endswith('.mid')]
music_files.sort()
def play_music_in_order(music_files_list):
    for music_file in music_files_list:
        pygame.mixer.music.load(os.path.join(music_directory, music_file))
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():  # 等待音乐播放完成
            pygame.time.Clock().tick(10)
play_music_in_order(music_files)           
started = False
paused = False


In [15]:
normalized_coords = []
pixel_coords = []
lmList = []  


# 创建保存数据的目录
output_dir = "output_data_test"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


#使用数据库保存得到的坐标
conn = sqlite3.connect('hand_coordinates.db')
# 创建一个表
db = conn.cursor()
# 创建用于保存归一化坐标的表
db.execute('''
CREATE TABLE IF NOT EXISTS normalized_coordinates (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    x REAL,
    y REAL
);
''')

# 创建用于保存像素坐标的表
db.execute('''
CREATE TABLE IF NOT EXISTS pixel_coordinates (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    x INTEGER,
    y INTEGER
);
''')
conn.commit()

In [17]:
# 初始化
cap = cv2.VideoCapture(0)

started = False
paused = False
#while True:
    # 音乐播放的检查
    #if not started and keyboard.is_pressed('s'):
        pygame.mixer.music.play()
        started = True
        while keyboard.is_pressed('s'):  # 等待直到释放's'键，避免多次触发
            pass
    #elif started and keyboard.is_pressed('p'):
        #if paused:
            pygame.mixer.music.unpause()
       # else:
            pygame.mixer.music.pause()
       # paused = not paused
        #while keyboard.is_pressed('p'):  # 等待直到释放'p'键，避免多次触发
         #   pass


    # 视频读取和预处理
    success, img = cap.read()
    if not success:
     print("Failed to capture image")
     continue 
    
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    print(results.multi_hand_landmarks)

    if results.multi_hand_landmarks:
        for handlms in results.multi_hand_landmarks:
            for index, lm in enumerate(handlms.landmark):
                if index == 8:  # 如果你只关心索引为8的关键点
                    h, w, c = img.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    print(index, cx, cy)
                    lmList.append([index, cx, cy])
                    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")

                    # 插入数据到数据库
                    db.execute("INSERT INTO normalized_coordinates (timestamp, x, y) VALUES (?, ?, ?)", (current_time, lm.x, lm.y))

                    # 将像素坐标保存到数据库
                    db.execute("INSERT INTO pixel_coordinates (timestamp, x, y) VALUES (?, ?, ?)", (current_time, cx, cy))

                    conn.commit()

                    # 在图像上画圆
                    cv2.circle(img, (cx, cy), 12, (0, 220, 23), cv2.FILLED)

            # 其他代码（绘制关键点等）

    # FPS显示和其他后处理
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 3)

    cv2.imshow('Image', img)

    if cv2.waitKey(1) & 0xFF == 27:
        break
        
    if keyboard.is_pressed('q'):  # 如果按下'q'键，结束程序
        break
    
#pygame.quit()
# 关闭数据库连接
conn.close()
# 释放摄像头资源
cap.release()
# 关闭所有 OpenCV 窗口
cv2.destroyAllWindows()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 24)

In [20]:
#娴熟数据
conn = sqlite3.connect('hand_coordinates.db')
db = conn.cursor()
# 查询数据库中的数据
db.execute("SELECT * FROM normalized_coordinates")

# 获取查询结果
rows = db.fetchall()

# 打印查询结果
for row in rows:
    print(row)


(1, '2023-10-05 22:04:02.216405', 0.6049511432647705, 0.8665966391563416)
(2, '2023-10-05 22:04:02.268642', 0.6110810041427612, 0.8994436860084534)
(3, '2023-10-05 22:04:02.492452', 0.06826428323984146, 0.4273393452167511)
(4, '2023-10-05 22:04:02.546154', 0.06385975331068039, 0.4310705065727234)
(5, '2023-10-05 22:04:02.817253', 0.4086558222770691, 0.2767074406147003)
(6, '2023-10-05 22:04:03.117286', 0.29430776834487915, 0.2600330114364624)
(7, '2023-10-05 22:04:03.123809', 0.8240323066711426, 0.4018878936767578)
(8, '2023-10-05 22:04:03.171349', 0.2930644154548645, 0.2623549699783325)
(9, '2023-10-05 22:04:03.178938', 0.823918879032135, 0.3943387567996979)
(10, '2023-10-05 22:04:03.379550', 0.5011753439903259, 0.336825430393219)
(11, '2023-10-05 22:04:03.431873', 0.4958139657974243, 0.34176552295684814)
(12, '2023-10-05 22:04:03.483613', 0.4511916935443878, 0.2697831392288208)
(13, '2023-10-05 22:04:03.536464', 0.4522232413291931, 0.26957809925079346)
(14, '2023-10-05 22:04:03.63245

In [21]:
#查看数据库
import pandas as pd

conn = sqlite3.connect('hand_coordinates.db')

# 将数据读入Pandas DataFrame
df = pd.read_sql_query("SELECT * from normalized_coordinates", conn)

# 显示数据
print(df)

      id                   timestamp         x         y
0      1  2023-10-05 22:04:02.216405  0.604951  0.866597
1      2  2023-10-05 22:04:02.268642  0.611081  0.899444
2      3  2023-10-05 22:04:02.492452  0.068264  0.427339
3      4  2023-10-05 22:04:02.546154  0.063860  0.431071
4      5  2023-10-05 22:04:02.817253  0.408656  0.276707
..   ...                         ...       ...       ...
531  532  2023-10-05 22:04:58.170215  0.529589  0.547509
532  533  2023-10-05 22:04:58.235467  0.659689  0.523536
533  534  2023-10-05 22:04:58.487984  0.315795  0.522262
534  535  2023-10-05 22:04:58.543565  0.312802  0.525732
535  536  2023-10-05 22:04:58.594797  0.278557  0.519068

[536 rows x 4 columns]
